In [1]:
from mlscorecheck.aggregated import (generate_dataset_specification,
                                        Dataset,
                                        solve)
import numpy as np

In [1]:
import datetime

from mlscorecheck.aggregated import (check_aggregated_scores,
                                        generate_experiment_specification,
                                        generate_dataset_specification,
                                        Experiment)

In [19]:
experiment_spec = {'aggregation': 'mor',
                        'datasets': [generate_dataset_specification(aggregation='mor',
                                                                    random_state=idx,
                                                                    max_n_folds=20,
                                                                    max_n_repeats=20)
                                        for idx in range(2)]}
print(datetime.datetime.now())
experiment = Experiment(**experiment_spec)
sample = experiment.sample(random_state=7)

print(datetime.datetime.now())

scores = sample.calculate_scores(rounding_decimals=7)

bounds = sample.get_dataset_fold_bounds(['acc', 'sens', 'spec', 'bacc'], True)
experiment = experiment.add_dataset_fold_bounds(bounds)

bounds = sample.get_dataset_bounds(['acc', 'sens', 'spec', 'bacc'], True)
experiment = experiment.add_dataset_bounds(bounds)

print(datetime.datetime.now())

flag, details = check_aggregated_scores(experiment.to_dict(problem_only=True),
                                        scores,
                                        1e-7,
                                        return_details=True,
                                        timeout=1)

print(datetime.datetime.now())

2023-08-27 00:11:17,656:INFO:creating a folding based on the specification
2023-08-27 00:11:17,658:INFO:querying p and n from looking up the dataset
2023-08-27 00:11:17,659:INFO:creating a folding based on the specification
2023-08-27 00:11:17,678:INFO:fold kiicdrkf_0_0: adding lower bound 0.11176470588235293 for acc
2023-08-27 00:11:17,679:INFO:fold kiicdrkf_0_0: adding upper bound 0.5117647058823529 for acc
2023-08-27 00:11:17,680:INFO:fold kiicdrkf_0_0: adding lower bound 0.11249999999999999 for sens
2023-08-27 00:11:17,681:INFO:fold kiicdrkf_0_0: adding upper bound 0.5125 for sens
2023-08-27 00:11:17,681:INFO:fold kiicdrkf_0_0: adding lower bound 0.1111111111111111 for spec
2023-08-27 00:11:17,682:INFO:fold kiicdrkf_0_0: adding upper bound 0.5111111111111111 for spec
2023-08-27 00:11:17,683:INFO:fold kiicdrkf_0_0: adding lower bound 0.11180555555555555 for bacc
2023-08-27 00:11:17,685:INFO:fold kiicdrkf_0_0: adding upper bound 0.5118055555555556 for bacc
2023-08-27 00:11:17,686:INF

2023-08-27 00:11:17.656169
2023-08-27 00:11:17.663514
2023-08-27 00:11:17.675473
asdf 1


2023-08-27 00:11:17,857:INFO:fold krclpryy_1_10: adding upper bound 1.0 for acc
2023-08-27 00:11:17,858:INFO:fold krclpryy_1_10: adding lower bound 0.8 for sens
2023-08-27 00:11:17,859:INFO:fold krclpryy_1_10: adding upper bound 1.0 for sens
2023-08-27 00:11:17,859:INFO:fold krclpryy_1_10: adding lower bound 0.6727272727272726 for spec
2023-08-27 00:11:17,859:INFO:fold krclpryy_1_10: adding upper bound 1.0 for spec
2023-08-27 00:11:17,860:INFO:fold krclpryy_1_10: adding lower bound 0.7363636363636363 for bacc
2023-08-27 00:11:17,860:INFO:fold krclpryy_1_10: adding upper bound 1.0 for bacc
2023-08-27 00:11:17,861:INFO:fold krclpryy_1_11: adding lower bound 0.14210526315789468 for acc
2023-08-27 00:11:17,862:INFO:fold krclpryy_1_11: adding upper bound 0.5421052631578946 for acc
2023-08-27 00:11:17,862:INFO:fold krclpryy_1_11: adding lower bound 0.8 for sens
2023-08-27 00:11:17,863:INFO:fold krclpryy_1_11: adding upper bound 1.0 for sens
2023-08-27 00:11:17,863:INFO:fold krclpryy_1_11: ad

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/291b70ac792c4d8faf5cb68b5adb4dae-pulp.mps sec 1 timeMode elapsed branch printingOptions all solution /tmp/291b70ac792c4d8faf5cb68b5adb4dae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 997 COLUMNS
At line 5839 RHS
At line 6832 BOUNDS
At line 7076 ENDATA
Problem MODEL has 992 rows, 243 columns and 4356 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 4 rows, 125 columns (125 integer (0 of which binary)) and 375 elements
Cbc0031I 4 added rows had average density of 109.75
Cbc0013I At root node, 4 cuts changed objective from 0 to 0 in 93 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 

In [20]:
details['lp_status']

'timeout'

In [2]:
import pulp as pl

In [56]:
pl.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

In [3]:
pl.listSolvers(onlyAvailable=True)

['GLPK_CMD', 'PULP_CBC_CMD']

In [51]:
solver = pl.getSolver('PULP_CBC_CMD', timeLimit=5)

In [52]:
score_subset = ['acc', 'sens', 'spec']
rounding_decimals = 3
random_state = 15
aggregation = 'mor'
TOL = 10**(-(rounding_decimals+1))

In [53]:
problem_orig = generate_dataset_specification(random_state=random_state,
                                        aggregation=aggregation)
problem = Dataset(**problem_orig)

sample = problem.sample(random_state)
scores = sample.calculate_scores(score_subset, rounding_decimals)
problem = problem.add_fold_bounds(sample.get_fold_bounds(score_subset, feasible=True))

print(problem_orig)
print(scores)
print(problem)

eps = 10**(-rounding_decimals)/2 + TOL

print(eps)

2023-08-26 03:36:04,691:INFO:generating a random id for the dataset
2023-08-26 03:36:04,692:INFO:creating a folding based on the specification


{'p': 758, 'n': 909, 'n_folds': 6, 'n_repeats': 1, 'aggregation': 'mor', 'identifier': None}
{'acc': 0.469, 'sens': 0.442, 'spec': 0.49}
{'identifier': 'bqbtiuusdywmqvgp', 'score_bounds': None, 'folds': [{'p': 126, 'n': 152, 'identifier': 'bqbtiuusdywmqvgp_0_0', 'score_bounds': {'acc': (0.3047553956834533, 0.763589928057554), 'sens': (-0.001, 0.9301338582677166), 'spec': (0.11158278145695365, 0.9220526315789473)}, 'figures': None}, {'p': 126, 'n': 152, 'identifier': 'bqbtiuusdywmqvgp_0_1', 'score_bounds': {'acc': (0.3047553956834533, 0.763589928057554), 'sens': (-0.001, 0.9301338582677166), 'spec': (0.11158278145695365, 0.9220526315789473)}, 'figures': None}, {'p': 126, 'n': 152, 'identifier': 'bqbtiuusdywmqvgp_0_2', 'score_bounds': {'acc': (0.3047553956834533, 0.763589928057554), 'sens': (-0.001, 0.9301338582677166), 'spec': (0.11158278145695365, 0.9220526315789473)}, 'figures': None}, {'p': 127, 'n': 151, 'identifier': 'bqbtiuusdywmqvgp_0_3', 'score_bounds': {'acc': (0.30475539568345

In [54]:
result = solve(problem, scores, eps, solver=solver)

2023-08-26 03:36:05,316:INFO:fold bqbtiuusdywmqvgp_0_0: adding lower bound 0.3047553956834533 for acc
2023-08-26 03:36:05,318:INFO:fold bqbtiuusdywmqvgp_0_0: adding upper bound 0.763589928057554 for acc
2023-08-26 03:36:05,322:INFO:fold bqbtiuusdywmqvgp_0_0: adding lower bound -0.001 for sens
2023-08-26 03:36:05,325:INFO:fold bqbtiuusdywmqvgp_0_0: adding upper bound 0.9301338582677166 for sens
2023-08-26 03:36:05,327:INFO:fold bqbtiuusdywmqvgp_0_0: adding lower bound 0.11158278145695365 for spec
2023-08-26 03:36:05,329:INFO:fold bqbtiuusdywmqvgp_0_0: adding upper bound 0.9220526315789473 for spec
2023-08-26 03:36:05,330:INFO:fold bqbtiuusdywmqvgp_0_1: adding lower bound 0.3047553956834533 for acc
2023-08-26 03:36:05,332:INFO:fold bqbtiuusdywmqvgp_0_1: adding upper bound 0.763589928057554 for acc
2023-08-26 03:36:05,333:INFO:fold bqbtiuusdywmqvgp_0_1: adding lower bound -0.001 for sens
2023-08-26 03:36:05,334:INFO:fold bqbtiuusdywmqvgp_0_1: adding upper bound 0.9301338582677166 for sens

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f32cc59461a24f86a1a7e6e0aa055cbd-pulp.mps sec 5 timeMode elapsed branch printingOptions all solution /tmp/f32cc59461a24f86a1a7e6e0aa055cbd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 47 COLUMNS
At line 169 RHS
At line 212 BOUNDS
At line 226 ENDATA
Problem MODEL has 42 rows, 13 columns and 96 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 5
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 9 rows, 12 columns (12 integer (0 of which binary)) and 36 elements
Cbc0031I 4 added rows had average density of 7.75
Cbc0013I At root node, 4 cuts changed objective from 0 to 0 in 32 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 0 c

In [55]:
result.status

0

In [8]:
scores

{'acc': 0.527, 'sens': 0.443, 'spec': 0.616}

In [9]:
problem.folds

[{'p': 59, 'n': 55, 'identifier': 'arifjscdmayhuxvn_0_0', 'score_bounds': {'acc': (0.15689473684210525, 0.9656017699115044), 'sens': (0.03289830508474576, 1.001), 'spec': (0.05354545454545454, 1.001)}, 'figures': None},
 {'p': 59, 'n': 55, 'identifier': 'arifjscdmayhuxvn_0_1', 'score_bounds': {'acc': (0.15689473684210525, 0.9656017699115044), 'sens': (0.03289830508474576, 1.001), 'spec': (0.05354545454545454, 1.001)}, 'figures': None},
 {'p': 59, 'n': 55, 'identifier': 'arifjscdmayhuxvn_0_2', 'score_bounds': {'acc': (0.15689473684210525, 0.9656017699115044), 'sens': (0.03289830508474576, 1.001), 'spec': (0.05354545454545454, 1.001)}, 'figures': None},
 {'p': 59, 'n': 55, 'identifier': 'arifjscdmayhuxvn_0_3', 'score_bounds': {'acc': (0.15689473684210525, 0.9656017699115044), 'sens': (0.03289830508474576, 1.001), 'spec': (0.05354545454545454, 1.001)}, 'figures': None},
 {'p': 59, 'n': 55, 'identifier': 'arifjscdmayhuxvn_0_4', 'score_bounds': {'acc': (0.15689473684210525, 0.96560176991150

In [10]:
for fold in sample.folds:
    print(fold.calculate_scores())

{'acc': 0.48245614035087714, 'sens': 0.6779661016949152, 'spec': 0.2727272727272727, 'bacc': 0.47534668721109397}
{'acc': 0.4649122807017544, 'sens': 0.7627118644067796, 'spec': 0.14545454545454545, 'bacc': 0.45408320493066257}
{'acc': 0.5701754385964912, 'sens': 0.3728813559322034, 'spec': 0.7818181818181817, 'bacc': 0.5773497688751925}
{'acc': 0.2543859649122807, 'sens': 0.3050847457627119, 'spec': 0.19999999999999998, 'bacc': 0.25254237288135595}
{'acc': 0.4122807017543859, 'sens': 0.6779661016949152, 'spec': 0.12727272727272726, 'bacc': 0.4026194144838212}
{'acc': 0.7345132743362832, 'sens': 0.5862068965517241, 'spec': 0.8909090909090909, 'bacc': 0.7385579937304074}
{'acc': 0.37168141592920356, 'sens': 0.5344827586206896, 'spec': 0.19999999999999998, 'bacc': 0.3672413793103448}
{'acc': 0.6017699115044248, 'sens': 0.3620689655172414, 'spec': 0.8545454545454545, 'bacc': 0.6083072100313479}
{'acc': 0.504424778761062, 'sens': 0.5254237288135594, 'spec': 0.48148148148148145, 'bacc': 0.5

In [11]:
import numpy as np
import pulp as pl

In [12]:
tmp = {'a': 5, 'b': 0}

In [13]:
(tmp | {'a': 6}) | {'a': 1}

{'a': 1, 'b': 0}

In [14]:
problem = pl.LpProblem('dummy')

In [15]:
var = pl.LpVariable('test', 0, 10, pl.LpInteger)

In [16]:
problem += var <= 20

In [17]:
len(problem.constraints)

1

In [18]:
var.lowBound

0

In [19]:
np.round(3.1415, 1)

3.1

In [20]:
from mlscorecheck.aggregated import (Fold, Dataset, Experiment,
                                        generate_dataset_specification,
                                        generate_experiment_specification)
from mlscorecheck.aggregated import solve
from mlscorecheck.experiments import dataset_statistics

In [21]:
TOL=1*1e-5
score_subset=['acc', 'bacc']
random_state=14
rounding_decimals=2
aggregation='rom'
aggregation_ds='mor'

In [22]:
problem = generate_experiment_specification(max_n_datasets=4,
                                                max_n_folds=3,
                                                max_n_repeats=2,
                                                random_state=random_state,
                                                aggregation=aggregation,
                                                aggregation_ds=aggregation_ds)
problem = Experiment(**problem)

sample = problem.sample(random_state)
problem = problem.add_dataset_bounds(sample.get_dataset_bounds(score_subset, feasible=True))

scores = sample.calculate_scores(score_subset, rounding_decimals)

eps = 10**(-rounding_decimals) + TOL

2023-08-26 03:30:30,802:INFO:generating a random id for the dataset
2023-08-26 03:30:30,805:INFO:creating a folding based on the specification
2023-08-26 03:30:30,807:INFO:generating a random id for the dataset
2023-08-26 03:30:30,809:INFO:a random identifier for the fold has been generated pdhtxacycdejshpa
2023-08-26 03:30:30,810:INFO:querying p and n from looking up the dataset
2023-08-26 03:30:30,812:INFO:creating a folding based on the specification
2023-08-26 03:30:30,813:INFO:generating a random id for the dataset
2023-08-26 03:30:30,815:INFO:creating a folding based on the specification


In [23]:
result = solve(problem, scores, eps)

2023-08-26 03:30:30,843:INFO:dataset amksgwfffzlqcazx: adding lower bound 0.3177514792899408 for acc
2023-08-26 03:30:30,846:INFO:dataset amksgwfffzlqcazx: adding upper bound 0.7177514792899409 for acc
2023-08-26 03:30:30,848:INFO:dataset amksgwfffzlqcazx: adding lower bound 0.3095416959601362 for bacc
2023-08-26 03:30:30,849:INFO:dataset amksgwfffzlqcazx: adding upper bound 0.7095416959601362 for bacc
2023-08-26 03:30:30,850:INFO:dataset pnskqkbswrthqjfb: adding lower bound 0.15664335664335666 for acc
2023-08-26 03:30:30,852:INFO:dataset pnskqkbswrthqjfb: adding upper bound 0.5566433566433566 for acc
2023-08-26 03:30:30,854:INFO:dataset pnskqkbswrthqjfb: adding lower bound 0.16210673038630025 for bacc
2023-08-26 03:30:30,855:INFO:dataset pnskqkbswrthqjfb: adding upper bound 0.5621067303863003 for bacc
2023-08-26 03:30:30,857:INFO:dataset common_datasets.ecoli_0_1_vs_2_3_5: adding lower bound 0.259016393442623 for acc
2023-08-26 03:30:30,858:INFO:dataset common_datasets.ecoli_0_1_vs_2_

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f1d7433ff3be4e25a04bd3b282f37082-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/f1d7433ff3be4e25a04bd3b282f37082-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 227 RHS
At line 248 BOUNDS
At line 270 ENDATA
Problem MODEL has 20 rows, 21 columns and 160 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 10 rows, 10 columns (10 integer (0 of which binary)) and 40 elements
Cbc0012I Integer solution of 0 found by DiveCoefficient after 3 iterations and 0 nodes (0.01 seconds)
Cbc0031I 3 added rows had average density of 4
Cbc0013I At root node, 3 cuts changed objective from 0 to 0 in 3 passes
Cbc0014I Cut gen

In [24]:
scores

{'acc': 0.37, 'bacc': 0.38}

In [25]:
sample.datasets[0].calculate_scores()

{'acc': 0.5177514792899408,
 'sens': 0.46727473742399117,
 'spec': 0.5518086544962812,
 'bacc': 0.5095416959601362}

In [26]:
sample.datasets[0].to_dict()

{'identifier': 'amksgwfffzlqcazx',
 'score_bounds': None,
 'folds': [{'p': 134,
   'n': 204,
   'identifier': 'amksgwfffzlqcazx_0_0',
   'score_bounds': None,
   'figures': {'tp': 107, 'tn': 88}},
  {'p': 135,
   'n': 203,
   'identifier': 'amksgwfffzlqcazx_0_1',
   'score_bounds': None,
   'figures': {'tp': 12, 'tn': 150}},
  {'p': 134,
   'n': 204,
   'identifier': 'amksgwfffzlqcazx_1_0',
   'score_bounds': None,
   'figures': {'tp': 71, 'tn': 102}},
  {'p': 135,
   'n': 203,
   'identifier': 'amksgwfffzlqcazx_1_1',
   'score_bounds': None,
   'figures': {'tp': 61, 'tn': 109}}],
 'aggregation': 'mor'}

In [27]:
sample.datasets[1].calculate_scores()

{'acc': 0.35664335664335667,
 'sens': 0.32098765432098764,
 'spec': 0.4032258064516129,
 'bacc': 0.36210673038630026}

In [28]:
problem.datasets[1].to_dict()

{'identifier': 'pnskqkbswrthqjfb',
 'score_bounds': {'acc': (0.15664335664335666, 0.5566433566433566),
  'bacc': (0.16210673038630025, 0.5621067303863003)},
 'folds': [{'p': 81,
   'n': 62,
   'identifier': 'pdhtxacycdejshpa',
   'score_bounds': None,
   'figures': None}],
 'aggregation': 'mor'}

In [29]:
sample.datasets[2].calculate_scores()

{'acc': 0.459016393442623,
 'sens': 0.35416666666666663,
 'spec': 0.4704545454545454,
 'bacc': 0.41231060606060604}

In [30]:
problem.datasets[2].to_dict()

{'identifier': 'common_datasets.ecoli_0_1_vs_2_3_5',
 'score_bounds': {'acc': (0.259016393442623, 0.659016393442623),
  'bacc': (0.21231060606060603, 0.612310606060606)},
 'folds': [{'p': 12,
   'n': 110,
   'identifier': 'common_datasets.ecoli_0_1_vs_2_3_5_0_0',
   'score_bounds': None,
   'figures': None},
  {'p': 12,
   'n': 110,
   'identifier': 'common_datasets.ecoli_0_1_vs_2_3_5_0_1',
   'score_bounds': None,
   'figures': None},
  {'p': 12,
   'n': 110,
   'identifier': 'common_datasets.ecoli_0_1_vs_2_3_5_1_0',
   'score_bounds': None,
   'figures': None},
  {'p': 12,
   'n': 110,
   'identifier': 'common_datasets.ecoli_0_1_vs_2_3_5_1_1',
   'score_bounds': None,
   'figures': None}],
 'aggregation': 'mor'}

In [31]:
experiment = Experiment(**generate_experiment_specification())

2023-08-26 03:30:31,040:INFO:creating a folding based on the specification
2023-08-26 03:30:31,041:INFO:a random identifier for the fold has been generated fzgdyygakpeenxad
2023-08-26 03:30:31,042:INFO:a random identifier for the fold has been generated hepyyzvrirhydlbd
2023-08-26 03:30:31,043:INFO:a random identifier for the fold has been generated opnpnmpsmlvvpbfn
2023-08-26 03:30:31,043:INFO:a random identifier for the fold has been generated wbwbscounhfkdwlv
2023-08-26 03:30:31,044:INFO:a random identifier for the fold has been generated tupafbfepeamjzkr
2023-08-26 03:30:31,045:INFO:a random identifier for the fold has been generated dalhbfwfgltsetxy
2023-08-26 03:30:31,046:INFO:a random identifier for the fold has been generated qekppywgqwekbors
2023-08-26 03:30:31,047:INFO:a random identifier for the fold has been generated kqdjuoiloqdihtpe
2023-08-26 03:30:31,048:INFO:a random identifier for the fold has been generated ltlinlolptkdmykf
2023-08-26 03:30:31,049:INFO:a random ident

In [32]:
sample = experiment.sample()

In [33]:
sample.calculate_scores()

{'acc': 0.4808915495216206,
 'sens': 0.4419621175327829,
 'spec': 0.5000797957229492,
 'bacc': 0.4710209566278661}

In [34]:
solve(experiment, sample.calculate_scores(), 1e-4).status == 1

2023-08-26 03:30:31,203:INFO:None: adding condition acc >= 0.480792
2023-08-26 03:30:31,205:INFO:None: adding condition acc <= 0.480992
2023-08-26 03:30:31,207:INFO:None: adding condition sens >= 0.441862
2023-08-26 03:30:31,208:INFO:None: adding condition sens <= 0.442062
2023-08-26 03:30:31,209:INFO:None: adding condition spec >= 0.499980
2023-08-26 03:30:31,211:INFO:None: adding condition spec <= 0.500180
2023-08-26 03:30:31,212:INFO:None: adding condition bacc >= 0.470921
2023-08-26 03:30:31,213:INFO:None: adding condition bacc <= 0.471121


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c2e9058322ac45278f38b5f5f4fb5b30-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/c2e9058322ac45278f38b5f5f4fb5b30-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 2207 RHS
At line 2216 BOUNDS
At line 2492 ENDATA
Problem MODEL has 8 rows, 275 columns and 1644 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 

True

In [35]:
dataset = Dataset(name='common_datasets.winequality-red-4', n_folds=2, n_repeats=3, aggregation='mor')

2023-08-26 03:30:31,242:INFO:querying p and n from looking up the dataset
2023-08-26 03:30:31,245:INFO:creating a folding based on the specification


In [36]:
ds2 = dataset.sample()

In [37]:
scores = ds2.calculate_scores()

In [38]:
res = solve(ds2, scores, 1e-4)

2023-08-26 03:30:31,283:INFO:common_datasets.winequality_red_4: adding condition acc >= 0.664691
2023-08-26 03:30:31,285:INFO:common_datasets.winequality_red_4: adding condition acc <= 0.664891
2023-08-26 03:30:31,286:INFO:common_datasets.winequality_red_4: adding condition sens >= 0.385465
2023-08-26 03:30:31,288:INFO:common_datasets.winequality_red_4: adding condition sens <= 0.385665
2023-08-26 03:30:31,289:INFO:common_datasets.winequality_red_4: adding condition spec >= 0.674329
2023-08-26 03:30:31,291:INFO:common_datasets.winequality_red_4: adding condition spec <= 0.674529
2023-08-26 03:30:31,292:INFO:common_datasets.winequality_red_4: adding condition bacc >= 0.529897
2023-08-26 03:30:31,293:INFO:common_datasets.winequality_red_4: adding condition bacc <= 0.530097


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6aa89e744dfb46798be5e3da319ca461-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/6aa89e744dfb46798be5e3da319ca461-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 111 RHS
At line 120 BOUNDS
At line 134 ENDATA
Problem MODEL has 8 rows, 13 columns and 72 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 2 tightened bounds,

In [39]:
res.status == 1

True

In [40]:
ds3 = dataset.populate(res)

In [41]:
ds3.folds[0].variable_names

{'tp': 'tp_common_datasets.winequality_red_4_0_0',
 'tn': 'tn_common_datasets.winequality_red_4_0_0'}

In [42]:
ds3.folds[0].populate(res)

{'p': 27, 'n': 773, 'identifier': 'common_datasets.winequality_red_4_0_0', 'score_bounds': None, 'figures': {'tn': 773.0, 'tp': 23.0}}

In [43]:
Dataset(**generate_dataset_specification())

2023-08-26 03:30:31,380:INFO:creating a folding based on the specification


{'identifier': 'xolavqpg', 'score_bounds': None, 'folds': [{'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_0', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_1', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_2', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_3', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_4', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_0_5', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 94, 'identifier': 'xolavqpg_0_6', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 94, 'identifier': 'xolavqpg_0_7', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_1_0', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_1_1', 'score_bounds': None, 'figures': None}, {'p': 46, 'n': 95, 'identifier': 'xolavqpg_1_2', 's

In [44]:
problem = Dataset(**generate_dataset_specification())

2023-08-26 03:30:31,398:INFO:generating a random id for the dataset
2023-08-26 03:30:31,400:INFO:a random identifier for the fold has been generated piaufhtdltibrlxd
2023-08-26 03:30:31,401:INFO:a random identifier for the fold has been generated ldugdqvpydnppwel
2023-08-26 03:30:31,402:INFO:a random identifier for the fold has been generated feaqakbltjiargxx
2023-08-26 03:30:31,402:INFO:a random identifier for the fold has been generated yriqhbjmsnzjpuzu
2023-08-26 03:30:31,403:INFO:a random identifier for the fold has been generated jyrcnpvtdrfvdabr
2023-08-26 03:30:31,405:INFO:a random identifier for the fold has been generated ljrdnxmhkbyuizqh
2023-08-26 03:30:31,406:INFO:a random identifier for the fold has been generated yghalkrtnbcntwln
2023-08-26 03:30:31,407:INFO:a random identifier for the fold has been generated ndemafwrqqewwdna
2023-08-26 03:30:31,407:INFO:a random identifier for the fold has been generated vepgzkhtjfnvpbxl
2023-08-26 03:30:31,409:INFO:a random identifier f

In [45]:
sample = problem.sample()

In [46]:
sample.calculate_figures()

{'tp': 705, 'tn': 534, 'p': 1513, 'n': 1060}

In [47]:
sample.calculate_scores()

{'acc': 0.47448535476679216,
 'sens': 0.4649938256311379,
 'spec': 0.46308416997699586,
 'bacc': 0.4640389978040668}

In [48]:
res = solve(problem.set_fold_bounds(sample.get_fold_bounds(feasible=False)), sample.calculate_scores(), 1e-4)

AttributeError: 'Dataset' object has no attribute 'set_fold_bounds'

In [ ]:
res.status == 1

False

In [ ]:
sample.get_bounds()

{'acc': (0.5925483870967742, 0.5945483870967742),
 'sens': (0.6786875, 0.6806875),
 'spec': (0.5701382113821138, 0.5721382113821138),
 'bacc': (0.624412855691057, 0.626412855691057)}

In [ ]:
problem.populate(res).calculate_scores()

{'acc': 0.5935483870967742,
 'sens': 0.6796875,
 'spec': 0.5711382113821138,
 'bacc': 0.625412855691057}

In [ ]:
dataset.calculate_scores()

NameError: name 'dataset' is not defined

In [ ]:
dataset = Dataset(p=5, n=10, n_folds=2, n_repeats=3, aggregation='mor')
dataset2 = Dataset(p=8, n=3, n_folds=2, n_repeats=3, aggregation='mor')
experiment = Experiment(datasets=[{'p': 5, 'n': 10, 'n_folds': 2, 'n_repeats': 3, 'aggregation': 'mor'},
                                    {'p': 8, 'n': 20, 'n_folds': 3, 'n_repeats': 3, 'aggregation': 'mor'}],
                        aggregation='mor')

2023-08-24 12:47:30,940:INFO:generating a random id for the dataset
2023-08-24 12:47:30,941:INFO:creating a folding based on the specification
2023-08-24 12:47:30,942:INFO:generating a random id for the dataset
2023-08-24 12:47:30,944:INFO:creating a folding based on the specification
2023-08-24 12:47:30,944:INFO:generating a random id for the dataset
2023-08-24 12:47:30,945:INFO:creating a folding based on the specification
2023-08-24 12:47:30,946:INFO:generating a random id for the dataset
2023-08-24 12:47:30,947:INFO:creating a folding based on the specification


In [ ]:
experiment.sample()

{'id': None, 'aggregation': 'mor', 'datasets': [{'id': 'cninryupyaqqxvnf', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 3, 'n': 5, 'id': 'cninryupyaqqxvnf_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 1, 'tn': 4}, 'scores': {'acc': 0.625, 'sens': 0.3333333333333333, 'spec': 0.8, 'npv': 0.6666666666666666, 'ppv': 0.5, 'bacc': 0.5666666666666667, 'f1p': 0.4, 'fm': 0.4082482904638631, 'p': 3, 'n': 5, 'tp': 1, 'tn': 4}}, {'p': 2, 'n': 5, 'id': 'cninryupyaqqxvnf_0_1', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 0, 'tn': 0}, 'scores': {'acc': 0.0, 'sens': 0.0, 'spec': 0.0, 'npv': 0.0, 'ppv': 0.0, 'bacc': 0.0, 'f1p': 0.0, 'fm': 0.0, 'p': 2, 'n': 5, 'tp': 0, 'tn': 0}}, {'p': 3, 'n': 5, 'id': 'cninryupyaqqxvnf_1_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 2, 'tn': 0}, 'scores': {'acc': 0.25, 'sens': 0.6666666666666666, 'spec': 0.0, 'npv': 0.0, 'ppv': 0.2857142857142857, 'bacc': 0.3333333333333333, 'f1p': 0.4, 'fm': 0.43643

In [ ]:
experiment.scores

{'acc': 0.4276895943562611,
 'sens': 0.4444444444444444,
 'spec': 0.43492063492063493,
 'bacc': 0.4396825396825397}

In [ ]:
res = solve(experiment, experiment.scores, eps=1e-4)

2023-08-24 12:47:30,987:INFO:None: adding condition acc >= 0.4275895943562611
2023-08-24 12:47:30,989:INFO:None: adding condition acc <= 0.42778959435626107
2023-08-24 12:47:30,990:INFO:None: adding condition sens >= 0.44434444444444443
2023-08-24 12:47:30,992:INFO:None: adding condition sens <= 0.4445444444444444
2023-08-24 12:47:30,993:INFO:None: adding condition spec >= 0.43482063492063494
2023-08-24 12:47:30,993:INFO:None: adding condition spec <= 0.4350206349206349
2023-08-24 12:47:30,994:INFO:None: adding condition bacc >= 0.4395825396825397
2023-08-24 12:47:30,995:INFO:None: adding condition bacc <= 0.43978253968253966


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a4ab4210a9304f23b32828ea9354afd9-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/a4ab4210a9304f23b32828ea9354afd9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 269 RHS
At line 278 BOUNDS
At line 309 ENDATA
Problem MODEL has 8 rows, 30 columns and 180 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12.9964 - 0.00 seconds
Cgl0004I processed model has 4 rows, 10 columns (10 integer (0 of which binary)) and 30 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0031I 2 added rows had average density of 7.5
Cbc0013I At root node, 2 cuts changed objective from 12.9964 to 13 in 13 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 

In [ ]:
Experiment(**experiment.to_dict(problem_only=True)).populate(res).scores

{'acc': 0.4276785714285714,
 'sens': 0.4444444444444445,
 'spec': 0.43492063492063493,
 'bacc': 0.4396825396825397}

In [ ]:
experiment.extract_bounds()

({'acc': [0.0, 0.8888888888888888],
  'sens': [0.0, 1.0],
  'spec': [0.0, 1.0],
  'bacc': [0.0, 0.9285714285714286]},
 {'tp': [0, 3], 'tn': [0, 6]})

In [ ]:
experiment.check_bounds()

{'id': None,
 'figures': {'tp': 17, 'tn': 39, 'p': 39, 'n': 90},
 'scores': {'acc': 0.4276895943562611,
  'sens': 0.4444444444444444,
  'spec': 0.43492063492063493,
  'bacc': 0.4396825396825397},
 'all_bounds': False,
 'datasets': [{'id': 'cninryupyaqqxvnf',
   'figures': {'tp': 5, 'tn': 13, 'p': 15, 'n': 30},
   'scores': {'acc': 0.4047619047619048,
    'sens': 0.3333333333333333,
    'spec': 0.43333333333333335,
    'bacc': 0.3833333333333333},
   'score_bounds': None,
   'check_score_bounds': None,
   'tptn_bounds': None,
   'check_tptn_bounds': None,
   'all_bounds': False,
   'folds': [{'id': 'cninryupyaqqxvnf_0_0',
     'figures': {'tp': 1, 'tn': 4},
     'scores': {'acc': 0.625,
      'sens': 0.3333333333333333,
      'spec': 0.8,
      'npv': 0.6666666666666666,
      'ppv': 0.5,
      'bacc': 0.5666666666666667,
      'f1p': 0.4,
      'fm': 0.4082482904638631,
      'p': 3,
      'n': 5,
      'tp': 1,
      'tn': 4},
     'score_bounds': None,
     'check_score_bounds': None

In [ ]:
experiment = Experiment(datasets=[{'name': 'common_datasets.ADA', 'aggregation': 'mor'},
                                    {'name': 'common_datasets.ecoli1', 'n_folds': 5, 'n_repeats': 3, 'aggregation': 'mor'}],
                        aggregation='mor')

2023-08-24 12:47:31,130:INFO:querying p and n from looking up the dataset
2023-08-24 12:47:31,132:INFO:creating a folding based on the specification
2023-08-24 12:47:31,133:INFO:querying p and n from looking up the dataset
2023-08-24 12:47:31,133:INFO:creating a folding based on the specification


In [ ]:
experiment.sample()

{'id': None, 'aggregation': 'mor', 'datasets': [{'id': 'common_datasets.ADA', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 1029, 'n': 3118, 'id': 'ADA_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 703, 'tn': 833}, 'scores': {'acc': 0.370388232457198, 'sens': 0.6831875607385811, 'spec': 0.2671584348941629, 'npv': 0.7187230371009491, 'ppv': 0.2352744310575636, 'bacc': 0.475172997816372, 'f1p': 0.35001244709982576, 'fm': 0.40091964863096236, 'p': 1029, 'n': 3118, 'tp': 703, 'tn': 833}}], 'aggregation': 'mor', 'figures': {'tp': 703, 'tn': 833, 'p': 1029, 'n': 3118}, 'scores': {'acc': 0.370388232457198, 'sens': 0.6831875607385811, 'spec': 0.2671584348941629, 'bacc': 0.475172997816372}}, {'id': 'common_datasets.ecoli1', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 16, 'n': 52, 'id': 'ecoli1_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 13, 'tn': 11}, 'scores': {'acc': 0.35294117647058826, 'sens': 0.8125, 'spec': 0.21153846

In [ ]:
experiment.calculate_scores()

{'acc': 0.4059506277884468,
 'sens': 0.6349271137026239,
 'spec': 0.33296333006648815,
 'bacc': 0.4839452218845561}

In [ ]:
experiment.extract_bounds()

({'acc': [0.08955223880597014, 0.7352941176470589],
  'sens': [0.2, 1.0],
  'spec': [0.019230769230769232, 0.8846153846153846],
  'bacc': [0.1446078431372549, 0.8173076923076923]},
 {'tp': [3, 703], 'tn': [1, 833]})

In [ ]:
scores = experiment.calculate_scores()

In [ ]:
scores

{'acc': 0.4059506277884468,
 'sens': 0.6349271137026239,
 'spec': 0.33296333006648815,
 'bacc': 0.4839452218845561}

In [ ]:
res = solve(experiment, scores, {'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

2023-08-24 12:47:31,212:INFO:None: adding condition acc >= 0.4058506277884468
2023-08-24 12:47:31,215:INFO:None: adding condition acc <= 0.4060506277884468
2023-08-24 12:47:31,216:INFO:None: adding condition sens >= 0.6348271137026239
2023-08-24 12:47:31,217:INFO:None: adding condition sens <= 0.6350271137026239
2023-08-24 12:47:31,219:INFO:None: adding condition spec >= 0.33286333006648816
2023-08-24 12:47:31,220:INFO:None: adding condition spec <= 0.33306333006648814
2023-08-24 12:47:31,221:INFO:None: adding condition bacc >= 0.4838452218845561
2023-08-24 12:47:31,223:INFO:None: adding condition bacc <= 0.48404522188455607


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c71d90ef7145457d9c5f4034c61a7968-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/c71d90ef7145457d9c5f4034c61a7968-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 286 RHS
At line 295 BOUNDS
At line 328 ENDATA
Problem MODEL has 8 rows, 32 columns and 192 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 508.474 - 0.00 seconds
Cgl0004I processed model has 4 rows, 8 columns (8 integer (0 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 512 found by DiveCoefficient after 14 iterations and 0 nodes (0.00 seconds)
Cbc0031I 3 added rows had average density of 4.3333333
Cbc0013I At root node, 3 cu

In [ ]:
experiment2 = Experiment(**experiment.to_dict(raw_problem=True))

TypeError: Experiment.to_dict() got an unexpected keyword argument 'raw_problem'

In [ ]:
experiment2.populate_with_solution(res)

In [ ]:
experiment2.calculate_scores()

{'acc': 0.39800535613256993,
 'sens': 0.7194298671849693,
 'spec': 0.2903846153846154,
 'bacc': 0.5049072412847924}

In [ ]:
experiment2.check_bounds()

{'figures': {'tp': 922.0, 'tn': 453.0, 'p': 1260, 'n': 3895},
 'scores': {'acc': 0.39800535613256993,
  'sens': 0.7194298671849693,
  'spec': 0.2903846153846154,
  'bacc': 0.5049072412847924},
 'downstream_bounds': True,
 'datasets': [{'figures': {'tp': 758.0, 'tn': 0.0, 'p': 1029, 'n': 3118},
   'scores': {'acc': 0.18278273450687244,
    'sens': 0.7366375121477162,
    'spec': 0.0,
    'bacc': 0.3683187560738581},
   'score_bounds': None,
   'check_score_bounds': True,
   'tptn_bounds': None,
   'check_tptn_bounds': True,
   'downstream_bounds': True,
   'folds': [{'figures': {'tn': 0.0, 'tp': 758.0},
     'scores': {'acc': 0.18278273450687244,
      'sens': 0.7366375121477162,
      'spec': 0.0,
      'npv': 0.0,
      'ppv': 0.195562435500516,
      'bacc': 0.3683187560738581,
      'f1p': 0.309072375127421,
      'fm': 0.379550557839991,
      'p': 1029,
      'n': 3118,
      'tn': 0.0,
      'tp': 758.0},
     'score_bounds': None,
     'check_score_bounds': True,
     'tptn_boun

In [ ]:
#import pulp as pl
#lp_program = pl.LpProblem('feasibility')

In [ ]:
#tp = pl.LpVariable('tp', 0, 5, pl.LpInteger)
#tn = pl.LpVariable('tn', 0, 10, pl.LpInteger)

In [ ]:
#lp_program += tp

In [ ]:
#lp_program

In [ ]:
#lp_program += tp <= 5

In [ ]:
#lp_program

In [ ]:
dataset = Dataset(name='common_datasets.ADA', n_repeats=2, n_folds=3, aggregation='rom')

2023-08-24 00:26:13,357:INFO:querying p and n from looking up the dataset
2023-08-24 00:26:13,358:INFO:creating a folding based on the specification


In [ ]:
dataset.simulate()

AttributeError: 'Dataset' object has no attribute 'simulate'

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2 = Dataset(**dataset.to_dict())

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2.aggregation = 'mor'

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.scores

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.linear_programming

In [ ]:
res = solve(dataset, dataset2.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

aaa
bbb
{'acc': 0.4115200984736789, 'sens': 0.3542274052478134, 'spec': 0.43045704696330295, 'bacc': 0.3923422261055582}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0, 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0, 'p': 2058, 'n': 6236, 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0, 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 

In [ ]:
dataset2.populate_with_solution(res)

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4114809454896286,
 'sens': 0.3542274052478134,
 'spec': 0.43038177735001604,
 'bacc': 0.3923045912989147}

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset.linear_programming

{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0,
 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0,
 'p': 2058,
 'n': 6236,
 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0,
 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 + 0.00048590864917395527*tp_ADA_1_1 + 0.00048590864917395527*tp_ADA_1_2 + 0.0,
 'spec': 0.00016035920461834508*tn_ADA_0_0 + 0.00016035920461834508*tn_ADA_0_1 + 0.000160359204618345

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
fold = Fold(p=5, n=10, id='fold_0')

In [ ]:
fold2 = Fold(**fold.to_dict(True))

In [ ]:
fold2.to_dict()

{'p': 5,
 'n': 10,
 'id': 'fold_0',
 'score_bounds': None,
 'tptn_bounds': None,
 'evaluation': None,
 'scores': None}

In [ ]:
fold.simulate()

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.evaluation

{'tp': 0.0, 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.p, fold.n

(5, 10)

In [ ]:
res = solve(fold, fold.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

{'acc': 0.13333333333333333, 'sens': 0.0, 'spec': 0.2, 'npv': 0.2857142857142857, 'ppv': 0.0, 'bacc': 0.1, 'f1p': 0.0, 'fm': 0.0, 'p': 5, 'n': 10, 'tp': 0.0, 'tn': 2.0}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': tp_fold_0, 'tn': tn_fold_0, 'acc': 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0 + 0.0, 'sens': 0.2*tp_fold_0 + 0.0, 'spec': 0.1*tn_fold_0 + 0.0, 'bacc': 0.05*tn_fold_0 + 0.1*tp_fold_0 + 0.0}
0.13333333333333333 0.0001 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0
0.0 0.0001 0.2*tp_fold_0
0.2 0.0001 0.1*tn_fold_0
0.1 0.0001 0.05*tn_fold_0 + 0.1*tp_fold_0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.sol (default strategy 1)
At line 2

In [ ]:
res.status

1

In [ ]:
res.variables()

[tn_fold_0, tp_fold_0]

In [ ]:
fold.populate_with_solution(res).calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}

In [ ]:
fold.evaluation

{'tn': 2.0, 'tp': 0.0}

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}